In [753]:
import numpy as np
import math
import scipy.stats as st
import plotly.express as px

In [754]:
def simulador(simulation_total_time, mean_arrival_time, mean_departure_time):
    n, number_of_arrivals, waiting_line_time, system_spent_time, clients_in_system, clients_in_line, last_event_time = 0, 0, 0, 0, 0, 0, 0
    queue, line_queue, arrivals, event = [], [], [], [0,""]
    queue.append([np.random.exponential(scale=1/mean_arrival_time), "Chegada"])
    while(event[0] < simulation_total_time):
        last_event_time, event = event[0], queue.pop(0)
        clients_in_system, clients_in_line = clients_in_system + ((event[0]-last_event_time) * n), clients_in_line + ((event[0]-last_event_time) * len (line_queue))
        if event[1] == "Chegada": 
            queue.append([event[0] + np.random.exponential(scale=1/mean_arrival_time), "Chegada"])
            arrivals.append(event[0])
            n, number_of_arrivals = n + 1, number_of_arrivals + 1
            if (n == 1):
                queue.append([event[0] + np.random.exponential(scale=1/mean_departure_time), "Saída"])
            if (n > 1):
                line_queue.append(event[0])
        else:
            if (len(line_queue) > 0):
                waiting_line_time += event[0] - line_queue.pop(0)
            n, system_spent_time = n - 1, system_spent_time + (event[0] - arrivals.pop(0)) 
            if (n > 0):
                queue.append([event[0] + np.random.exponential(scale=1/mean_departure_time), "Saída"])
        queue.sort()
    return n, event[0], waiting_line_time/number_of_arrivals, system_spent_time/number_of_arrivals, number_of_arrivals, clients_in_system/event[0], clients_in_line/event[0]

In [755]:
total_simulation_time = 100000
mean_arrival_time = 1
mean_departure_time = 4

In [756]:
n, simul_time, mean_waiting_inline_time, mean_system_spent_time, number_of_arrivals, mean_clients_in_system, mean_clients_in_line = simulador(total_simulation_time, mean_arrival_time, mean_departure_time)

In [757]:
print(f"\nTotal de pessoas que foram ao banco na simulação: {number_of_arrivals}")
print(f"Número de pessoas na fila ao final da simulação: {n}")
print(f"Tempo total da simulação: {simul_time:.4f}")
print(f"Tempo médio do cliente no sistema: {(mean_system_spent_time):.4f}")
print(f"Tempo médio de espera na fila: {(mean_waiting_inline_time):.4f}")
print(f"Número médio de clientes no sistema: {(mean_clients_in_system):.4f}")
print(f"Número médio de clientes na fila: {(mean_clients_in_line):.4f}")


Total de pessoas que foram ao banco na simulação: 99665
Número de pessoas na fila ao final da simulação: 1
Tempo total da simulação: 100000.1821
Tempo médio do cliente no sistema: 0.3349
Tempo médio de espera na fila: 0.0846
Número médio de clientes no sistema: 0.3338
Número médio de clientes na fila: 0.0843


In [758]:
def simulator_confidence_interval(sample_size, confidence_level):
    """
    Simula o processo de fila m/m/1 um intervalo de confiança para tempo médio de espera na fila.

    sample_size: tamanho da amostra para a simulação
    confidence_level: nível de confiança desejado para o intervalo de confiança

    Retorna um intervalo de confiança para o tempo médio de espera na fila.
    """
    results = [simulador(1000, 1, 2)[2] for _ in range(sample_size)]


    # Cálculo do intervalo de confiança
    # proportion = np.sum(results) / sample_size
    # std_error = math.sqrt(proportion * (1 - proportion) / sample_size)
    # margin_error = std_error * st.norm.ppf(confidence_level)
    # lower_bound = proportion - margin_error #limite superior do IC
    # upper_bound = proportion + margin_error  # limite inferior do IC


    # Outro calculo
    x_barra = np.sum(results) / sample_size
    z = 1.96
    s = np.std(results)
    limite_inferior = x_barra - z*s/math.sqrt(sample_size)
    limite_superior = x_barra + z*s/math.sqrt(sample_size)

    # print(limite_inferior, limite_superior)
    return (limite_inferior, limite_superior), results
    return (lower_bound, upper_bound), results

intervalo, data = simulator_confidence_interval(1000, 0.95)

print(intervalo)

(0.4938534644694335, 0.5044504317890833)


In [759]:
px.histogram(data, nbins=50)

## Avaliando Resultados

Link para referência: https://people.revoledu.com/kardi/tutorial/Queuing/MM1-Queuing-System.html

W = tempo médio que o cliente permanece no sistema (tempo de espera na fila + tempo de serviço). ${W = \frac{1}{\mu - \lambda}}$.\
Wq = tempo médio que os clientes devem esperar na fila até serem atendidos. ${Wq = W - \frac{1}{\mu}}$.\
L = número médio de clientes no sistema (na fila e sendo servidos). ${L = \lambda W}$\
Lq = número médio de clientes na fila. ${L_q = \lambda W_q}$

### Caso ${\lambda} = 1$ e ${\mu} = 2$

*Tomando uma simulação como exemplo, temos a seguinte saída:*

Total de pessoas que foram ao banco na simulação: ``100175``\
Número de pessoas na fila ao final da simulação: ``1``\
Tempo total da simulação: ``100001.0090``\
Tempo médio do cliente no sistema: ``1.0027``\
Tempo médio de espera na fila: ``0.5001``\
Número médio de clientes no sistema: ``1.0045``\
Número médio de clientes na fila: ``0.5010``

Como vimos mais acima:
${W = \frac{1}{\mu - \lambda}}$. Logo, com ${\lambda = 1}$ e ${\mu = 2}$ temos: ${W = \frac{1}{2 - 1}}$ = 1\
${W_q = W - \frac{1}{\mu}}$. Logo, com ${W= 1}$ e ${\mu = 2}$ temos: ${W_q = 1 - \frac{1}{2}}$ = ${\frac{1}{2}}$\
${L = \lambda W}$. Logo, com ${W= 1}$ e ${\lambda = 1}$ temos: ${L = 1}$\
${L_q = \lambda W_q}$. Logo, com ${W_q= \frac{1}{2}}$ e ${\lambda = 1}$ temos: ${L_q = \frac{1}{2}}$

Os resultados condizem com os valores da simulação!!

### Caso ${\lambda} = 2$ e ${\mu} = 4$

*Tomando uma simulação como exemplo, temos a seguinte saída:*

Total de pessoas que foram ao banco na simulação: ``199062``\
Número de pessoas na fila ao final da simulação: ``0``\
Tempo total da simulação: ``100000.0340``\
Tempo médio do cliente no sistema: ``0.4973``\
Tempo médio de espera na fila: ``0.2475``\
Número médio de clientes no sistema: ``0.9898``\
Número médio de clientes na fila: ``0.4926``

Como vimos mais acima:
${W = \frac{1}{\mu - \lambda}}$. Logo, com ${\lambda = 2}$ e ${\mu = 4}$ temos: ${W = \frac{1}{4 - 2}} = \frac{1}{2}$\
${W_q = W - \frac{1}{\mu}}$. Logo, com ${W= \frac{1}{2}}$ e ${\mu = 4}$ temos: ${W_q = \frac{1}{2} - \frac{1}{4}}$ = ${\frac{1}{4}}$\
${L = \lambda W}$. Logo, com ${W= \frac{1}{2}}$ e ${\lambda = 2}$ temos: ${L = 2\frac{1}{2} = 1}$\
${L_q = \lambda W_q}$. Logo, com ${W_q= \frac{1}{4}}$ e ${\lambda = 2}$ temos: ${L_q = \frac{2}{4} = \frac{1}{2}}$

Os resultados condizem com os valores da simulação!!

### Caso ${\lambda} = 1$ e ${\mu} = 4$

*Tomando uma simulação como exemplo, temos a seguinte saída:*

Total de pessoas que foram ao banco na simulação: ``100563``\
Número de pessoas na fila ao final da simulação: ``1``\
Tempo total da simulação: ``100001.2990``\
Tempo médio do cliente no sistema: ``0.3329``\
Tempo médio de espera na fila: ``0.0832``\
Número médio de clientes no sistema: ``0.3348``\
Número médio de clientes na fila: ``0.0837``

Como vimos mais acima:
${W = \frac{1}{\mu - \lambda}}$. Logo, com ${\lambda = 1}$ e ${\mu = 4}$ temos: ${W = \frac{1}{4 - 1}} = \frac{1}{3}$\
${W_q = W - \frac{1}{\mu}}$. Logo, com ${W= \frac{1}{3}}$ e ${\mu = 4}$ temos: ${W_q = \frac{1}{3} - \frac{1}{4}}$ = ${\frac{1}{12}}$\
${L = \lambda W}$. Logo, com ${W= \frac{1}{3}}$ e ${\lambda = 1}$ temos: ${L = \frac{1}{3}}$\
${L_q = \lambda W_q}$. Logo, com ${W_q= \frac{1}{12}}$ e ${\lambda = 1}$ temos: ${L_q = \frac{1}{12}}$

Os resultados condizem com os valores da simulação!!